# Augment and downscale data

In [1]:
import pathlib
import cv2
import numpy as np
import random
import shutil
from tqdm import tqdm

## Data folders

In [2]:
def create_clean_folder(path):
    path = pathlib.Path(path)
    path.mkdir(parents=True, exist_ok=True)
    shutil.rmtree(path)
    path.mkdir(parents=True, exist_ok=True)

data_path = pathlib.Path('../../data/')
rid_path = data_path / 'RID'
input_path = rid_path / 'input'
output_path = rid_path / 'output'

# Roooftop processed and split data paths
segment_dataset_path = output_path / 'masks_segments_reviewed'
segment_train_path = segment_dataset_path / 'train'
segment_train_image_path = segment_train_path / 'image'
segment_train_label_path = segment_train_path / 'label'
segment_valz19_path = segment_dataset_path / 'val_z19'
segment_valz19_image_path = segment_valz19_path / 'image'
create_clean_folder(segment_valz19_image_path)
segment_valz19_label_path = segment_valz19_path / 'label'
create_clean_folder(segment_valz19_label_path)

# Rooftop augmented data paths
segment_aug_dataset_path = output_path / 'masks_segments_reviewed_aug'
segment_aug_train_path = segment_aug_dataset_path / 'train'
segment_aug_train_image_path = segment_aug_train_path / 'image'
create_clean_folder(segment_aug_train_image_path)
segment_aug_train_label_path = segment_aug_train_path / 'label'
create_clean_folder(segment_aug_train_label_path)

# Superstructure rocessed and split data paths
superstructures_dataset_path = output_path / 'masks_superstructures_reviewed'
superstructures_train_path = superstructures_dataset_path / 'train'
superstructures_train_image_path = superstructures_train_path / 'image'
superstructures_train_label_path = superstructures_train_path / 'label'

# Superstructure augmented data paths
superstructures_aug_dataset_path = output_path / 'masks_superstructures_reviewed_aug'
superstructures_aug_train_path = superstructures_aug_dataset_path / 'train'
superstructures_aug_train_image_path = superstructures_aug_train_path / 'image'
create_clean_folder(superstructures_aug_train_image_path)
superstructures_aug_train_label_path = superstructures_aug_train_path / 'label'
create_clean_folder(superstructures_aug_train_label_path)

## Data augmentation

Transformations performed:

- Multiplying the sample data by a factor defined by AUGMENT_FACTOR
- Concatenate 4 images into one and resize them to downscale from 10cm/pixel to 20cm/pixel
- Randomly rotate by 0, 90, 180 or 270 degrees
- Randomly perform an average filter
- Randomly flip horizontally
- Randomly flip vertically

In [3]:
def multiply_list_float(l, factor):
    factor_int = int(factor)
    out = l * factor_int
    diff = factor - factor_int
    diff_len = int(diff * len(l))
    
    return out + l[:diff_len]

def concat_images(img1, img2, img3, img4, rescaling_factor=0.5):
    img_v = cv2.vconcat([img1, img2])
    img_h = cv2.vconcat([img3, img4])
    img = cv2.hconcat([img_v, img_h])
    
    return cv2.resize(img, (0,0), fx=rescaling_factor, fy=rescaling_factor)

def process_pair(img, mask, correct):
    # Randomly rotate by a multiply of 90, and correct pixels by corresponding value
    rotations = {
        None: 0,
        cv2.ROTATE_90_CLOCKWISE: 4,
        cv2.ROTATE_180: 8,
        cv2.ROTATE_90_COUNTERCLOCKWISE: 12
    }
    rotation, correction = random.choice(list(rotations.items()))
    
    if rotation:
        img = cv2.rotate(img, rotation)
        mask = cv2.rotate(mask, rotation)

    if correct:
        corrected = (mask + correction) % 16
        filtered = mask>16
        # where mask is greater than 16, keep the original mask, otherwise correct
        mask = np.where(filtered, mask, corrected)
    
    # Random average filtering every other image
    avg_filter = random.uniform(0, 1)
    if avg_filter > 0.5:
        kernel = np.ones((5,5),np.float32) / random.randrange(15,30)
        img = cv2.filter2D(img,-1,kernel)
    
    # Flip vertically every other image
    flip_filter = random.uniform(0, 1)
    if flip_filter:
        if correct:
            mask_tmp = np.copy(mask)
            mask_tmp[mask == 0] = 8
            mask_tmp[mask == 8] = 0
            mask_tmp[mask == 1] = 7
            mask_tmp[mask == 7] = 1
            mask_tmp[mask == 2] = 6
            mask_tmp[mask == 6] = 2
            mask_tmp[mask == 3] = 5
            mask_tmp[mask == 5] = 3
            mask_tmp[mask == 9] = 15
            mask_tmp[mask == 15] = 9
            mask_tmp[mask == 10] = 14
            mask_tmp[mask == 14] = 10
            mask_tmp[mask == 11] = 13
            mask_tmp[mask == 13] = 11
            mask = mask_tmp
        mask = cv2.flip(mask, 0)
        img = cv2.flip(img, 0)
        
    # Flip horizontally every other image
    flip_filter = random.uniform(0, 1)
    if flip_filter:
        if correct:
            mask_tmp = np.copy(mask)
            mask_tmp[mask == 1] = 15
            mask_tmp[mask == 15] = 1
            mask_tmp[mask == 2] = 14
            mask_tmp[mask == 14] = 2
            mask_tmp[mask == 3] = 13
            mask_tmp[mask == 13] = 3
            mask_tmp[mask == 4] = 12
            mask_tmp[mask == 12] = 4
            mask_tmp[mask == 5] = 11
            mask_tmp[mask == 11] = 5
            mask_tmp[mask == 6] = 10
            mask_tmp[mask == 10] = 6
            mask_tmp[mask == 7] = 9
            mask_tmp[mask == 9] = 7
            mask = mask_tmp
        mask = cv2.flip(mask, 1)
        img = cv2.flip(img, 1)
    
    return img, mask

## Processing Rooftop images at original size

In [4]:
# Copy original images and randomly transform them
path_list = list(segment_train_image_path.glob('*.png'))
random.shuffle(path_list)

for path in tqdm(path_list):
    name = path.name
    img = cv2.imread(str(segment_train_image_path / name))
    mask = cv2.imread(str(segment_train_label_path / name))

    cv2.imwrite(str(segment_aug_train_image_path / name), img)
    cv2.imwrite(str(segment_aug_train_label_path / name), mask)
    
    img, mask = process_pair(img, mask, correct=True)
    
    cv2.imwrite(str(segment_aug_train_image_path / name.replace('.png', '_transform.png')), img)
    cv2.imwrite(str(segment_aug_train_label_path / name.replace('.png', '_transform.png')), mask)

100%|██████████████████████████████████████████████████████████████████| 1504/1504 [01:56<00:00, 12.87it/s]


## Processing Rooftop images at half size

In [5]:
# Factor to augment the data, or how much to increase the original dataset
AUGMENT_FACTOR = 8
path_list = list(segment_train_image_path.glob('*.png'))
path_list = multiply_list_float(path_list, AUGMENT_FACTOR)

# Iterate every 4 images to get also lower resolution images
for i in tqdm(range(0, len(path_list), 4)):
    name_1 = path_list[i].name
    name_2 = path_list[i+1].name
    name_3 = path_list[i+2].name
    name_4 = path_list[i+3].name
    
    img_1 = cv2.imread(str(segment_train_image_path / name_1))
    img_2 = cv2.imread(str(segment_train_image_path / name_2))
    img_3 = cv2.imread(str(segment_train_image_path / name_3))
    img_4 = cv2.imread(str(segment_train_image_path / name_4))
    mask_1 = cv2.imread(str(segment_train_label_path / name_1))
    mask_2 = cv2.imread(str(segment_train_label_path / name_2))
    mask_3 = cv2.imread(str(segment_train_label_path / name_3))
    mask_4 = cv2.imread(str(segment_train_label_path / name_4))
    
    img = concat_images(img_1, img_2, img_3, img_4, 0.5)
    mask = concat_images(mask_1, mask_2, mask_3, mask_4, 0.5)

    # For z19 training validation
    cv2.imwrite(str(segment_valz19_image_path / (str(i) + '_concat.png')), img)
    cv2.imwrite(str(segment_valz19_label_path / (str(i) + '_concat.png')), mask)

    cv2.imwrite(str(segment_aug_train_image_path / (str(i) + '_concat.png')), img)
    cv2.imwrite(str(segment_aug_train_label_path / (str(i) + '_concat.png')), mask)
    
    img, mask = process_pair(img, mask, correct=True)
    
    cv2.imwrite(str(segment_aug_train_image_path / (str(i) + '_concat_transform.png')), img)
    cv2.imwrite(str(segment_aug_train_label_path / (str(i) + '_concat_transform.png')), mask)

100%|██████████████████████████████████████████████████████████████████| 3008/3008 [04:40<00:00, 10.73it/s]


## Processing Superstructure images at original size

In [6]:
# Copy original images and randomly transform them
path_list = list(superstructures_train_image_path.glob('*.png'))
random.shuffle(path_list)

for path in tqdm(path_list):
    name = path.name
    img = cv2.imread(str(superstructures_train_image_path / name))
    mask = cv2.imread(str(superstructures_train_label_path / name))

    cv2.imwrite(str(superstructures_aug_train_image_path / name), img)
    cv2.imwrite(str(superstructures_aug_train_label_path / name), mask)
    
    img, mask = process_pair(img, mask, correct=False)
    
    cv2.imwrite(str(superstructures_aug_train_image_path / name.replace('.png', '_transform.png')), img)
    cv2.imwrite(str(superstructures_aug_train_label_path / name.replace('.png', '_transform.png')), mask)

100%|██████████████████████████████████████████████████████████████████| 1504/1504 [01:30<00:00, 16.56it/s]


## Processing Superstructure images at half size

In [7]:
# Factor to augment the data, or how much to increase the original dataset
AUGMENT_FACTOR = 4
path_list = list(superstructures_train_image_path.glob('*.png'))
path_list = multiply_list_float(path_list, AUGMENT_FACTOR)

# Iterate every 4 images to get also lower resolution images
for i in tqdm(range(0, len(path_list), 4)):
    name_1 = path_list[i].name
    name_2 = path_list[i+1].name
    name_3 = path_list[i+2].name
    name_4 = path_list[i+3].name
    
    img_1 = cv2.imread(str(superstructures_train_image_path / name_1))
    img_2 = cv2.imread(str(superstructures_train_image_path / name_2))
    img_3 = cv2.imread(str(superstructures_train_image_path / name_3))
    img_4 = cv2.imread(str(superstructures_train_image_path / name_4))
    mask_1 = cv2.imread(str(superstructures_train_label_path / name_1))
    mask_2 = cv2.imread(str(superstructures_train_label_path / name_2))
    mask_3 = cv2.imread(str(superstructures_train_label_path / name_3))
    mask_4 = cv2.imread(str(superstructures_train_label_path / name_4))
    
    img = concat_images(img_1, img_2, img_3, img_4, 0.5)
    mask = concat_images(mask_1, mask_2, mask_3, mask_4, 0.5)

    cv2.imwrite(str(superstructures_aug_train_image_path / (str(i) + '_concat.png')), img)
    cv2.imwrite(str(superstructures_aug_train_label_path / (str(i) + '_concat.png')), mask)
    
    img, mask = process_pair(img, mask, correct=False)
    
    cv2.imwrite(str(superstructures_aug_train_image_path / (str(i) + '_concat_transform.png')), img)
    cv2.imwrite(str(superstructures_aug_train_label_path / (str(i) + '_concat_transform.png')), mask)

100%|██████████████████████████████████████████████████████████████████| 1504/1504 [01:44<00:00, 14.36it/s]
